In [22]:
import importlib
import cv2
import json
import os
import math
import copy
from _paths import nomeroff_net_dir

from nomeroff_net.pipes.number_plate_text_readers.base.ocr import OCR
from nomeroff_net.tools.mcm import modelhub

In [23]:
# auto download latest dataset
from nomeroff_net.tools import modelhub

# auto download latest dataset
#info = modelhub.download_dataset_for_model("EuUa1995")
#PATH_TO_DATASET = info["dataset_path"]
PATH_TO_DATASET = os.path.join(nomeroff_net_dir, "data/dataset/TextDetector/ocr_example")

In [24]:
def compare(model_path='latest',
           text_detector_name = "eu_ua_2004_2015",
           img_format = "png",
           root_dir=os.path.join(PATH_TO_DATASET, "test"),
           predicted_part_size=1,
           acc_less_than = 0.7,
           replace_tamplate = None,
           model_conf = None):
    if replace_tamplate is None:
        replace_tamplate = {'moderation': {'isModerated': 1, 'moderatedBy': 'ApelSYN'}}
    if model_conf is None:
        model_conf = copy.deepcopy(modelhub.models[text_detector_name])
    text_detector = OCR(model_name=text_detector_name, letters=model_conf["letters"],
                        linear_size=model_conf["linear_size"], max_text_len=model_conf["max_text_len"],
                        height=model_conf["height"], width=model_conf["width"],
                        color_channels=model_conf["color_channels"],
                        hidden_size=model_conf["hidden_size"], backbone=model_conf["backbone"])
    text_detector.init_label_converter()
    text_detector.load(model_path)

    ann_dir = os.path.join(root_dir, "ann")
    jsons = []
    jsons_paths = []
    for dir_name, subdir_list, file_list in os.walk(ann_dir):
        for fname in file_list:
            fname = os.path.join(ann_dir, fname)
            jsons_paths.append(fname)
            with open(fname) as jsonF:
                jsonData = json.load(jsonF)
            jsons.append(jsonData)
    print("LOADED {} ANNOTATIONS".format(len(jsons)))

    img_dir = os.path.join(root_dir, "img")
    imgs = []
    for j in jsons:
        img_path =os.path.join(img_dir, "{}.{}".format(j["name"], img_format))
        img = cv2.imread(img_path)
        imgs.append(img)
    print("LOADED {} IMAGES".format(len(imgs)))

    predicted = []
    accs      = []
    N = math.ceil(len(imgs) / predicted_part_size)
    for i in range(N):
        part = i*predicted_part_size
        part_imgs = imgs[part:part+predicted_part_size]
        xs = text_detector.preprocess(part_imgs)
        predicted_part, net_out_value_part = text_detector.predict(xs, return_acc=True)
        predicted += predicted_part


        # get accuracy
        if acc_less_than >= 1:
            # not process acc
            accs  += [1 for _predicted in predicted_part]
            continue
        # process accuracy
        acc_part = []
        for _predicted, _net_out_value in zip(predicted_part, net_out_value_part):
            acc = text_detector.get_acc([_net_out_value], [_predicted])
            acc_part.append(acc)
        accs  += acc_part


    print("PREDICTED {} IMAGES".format(len(predicted)))

    err_cnt = 0
    for i in range(len(jsons_paths)):
        json_path      = jsons_paths[i]
        predicted_item = predicted[i]
        jsonData       = jsons[i]
        acc            = accs[i]
        jsonData["moderation"]["predicted"] = predicted_item

        if jsonData["description"].lower() == jsonData["moderation"]["predicted"].lower() and acc > acc_less_than:
            jsonData["moderation"]["isModerated"] = 1
        else:
            print("Predicted '{}' with acc {}, real: '{}' in file {}".format(
                jsonData["moderation"]["predicted"].lower(),
                acc,
                jsonData["description"].lower(),
                json_path))
            err_cnt = err_cnt+1
            jsonData["moderation"]["isModerated"] = 0
        with open(json_path, "w", encoding='utf8') as jsonWF:
            json.dump(jsonData, jsonWF,  ensure_ascii=False)

    print("Error detection count: {}".format(err_cnt))
    print("Accuracy: {}".format(1-err_cnt/len(predicted)))

In [29]:
compare(model_path="latest", acc_less_than = 0.75)

LOADED 4 ANNOTATIONS
LOADED 4 IMAGES
PREDICTED 4 IMAGES
Predicted '379895b' with acc 0.9744253754615784, real: '0038sc' in file /mnt/data/var/www/nomeroff-net/examples/ju/dataset_tools/../../../data/dataset/TextDetector/ocr_example/test/ann/0038SC-0.json
Predicted '371kpk' with acc 0.8737720251083374, real: 'x371hk96' in file /mnt/data/var/www/nomeroff-net/examples/ju/dataset_tools/../../../data/dataset/TextDetector/ocr_example/test/ann/1000_X371HK96_0.json
Predicted '13bm' with acc 0.6171997785568237, real: 'aa0013bm' in file /mnt/data/var/www/nomeroff-net/examples/ju/dataset_tools/../../../data/dataset/TextDetector/ocr_example/test/ann/AA0013BM.json
Predicted '20146kc' with acc 0.9199544191360474, real: '2914 kc-7' in file /mnt/data/var/www/nomeroff-net/examples/ju/dataset_tools/../../../data/dataset/TextDetector/ocr_example/test/ann/10001_2914KC7_0.json
Error detection count: 4
Accuracy: 0.0


In [34]:
# # or load your own trained model
# compare(
#     model_path="../../../data/models/shufflenet_v2_x2_0/anpr_ocr_eu_2004_2015_2022_11_11_shufflenet_v2_x2_0.ckpt",
#     acc_less_than = 0.75,
#     model_conf={
#       "backbone": "shufflenet_v2_x2_0",
#       "letters": ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "A", "B", "C", "D", "E", "F", "G", "H",
#                   "I", "K", "M", "O", "P", "T", "X", "Y", "Z"],
#       "max_text_len": 8,
#       "height": 50,
#       "width": 200,
#       "color_channels": 3,
#       "bidirectional": True,
#       "hidden_size": 32,
#       "linear_size": 416
#     }
# )

LOADED 4 ANNOTATIONS
LOADED 4 IMAGES
PREDICTED 4 IMAGES
Predicted 'bp93bp' with acc 0.7681334018707275, real: '0038sc' in file /mnt/data/var/www/nomeroff-net/examples/ju/dataset_tools/../../../data/dataset/TextDetector/ocr_example/test/ann/0038SC-0.json
Predicted 'bh71p' with acc 0.8713468909263611, real: 'x371hk96' in file /mnt/data/var/www/nomeroff-net/examples/ju/dataset_tools/../../../data/dataset/TextDetector/ocr_example/test/ann/1000_X371HK96_0.json
Predicted 'a2916ct' with acc 0.9012768268585205, real: '2914 kc-7' in file /mnt/data/var/www/nomeroff-net/examples/ju/dataset_tools/../../../data/dataset/TextDetector/ocr_example/test/ann/10001_2914KC7_0.json
Error detection count: 3
Accuracy: 0.25
